In [1]:
from pulp import *
import numpy as np
from mpi_utils import *

### to add CLI 

In [153]:
mcpp = load_instances(1)


In [154]:
def And(model, a, b, name):
        """
        And(a,b)
        :param a: first parameter of And condition
        :param b: second parameter of And condition
        :param name: name of the And
        :return: 1 if a and b is true, false otherwise
        """
        delta = LpVariable(cat=LpInteger, name=name)
        model += delta <= a
        model += delta >= a + b - 1
        model += delta >= 0
        model += delta <= b
        return delta
def linear_prod(model, binary_var, countinuos_var, ub, name):
        """
        res = binary_var * countinuos_var
        :param binary_var: binary variable
        :param countinuos_var: countinuos variable
        :param ub: upper bound of the countinuos variable
        :param name: name of the product
        :return: the result of the product
        """
        res = LpVariable(cat=LpInteger, name=name)
        model += ub * binary_var >= res
        model += countinuos_var >= res
        model += countinuos_var - (1 - binary_var) * ub <= res
        model += res >= 0
        return res

In [155]:
# matrix k*m*l assigning the couriers with the packages, defining also their order
X  = [[[LpVariable(name = f'X_{i}_{j}_{k}', lowBound = 0, upBound = 1, cat = LpBinary) for k in range(0,mcpp.n + 1)] for j in range(mcpp.n)] for i in range(mcpp.m)]

#distance made by each courier
dist_courier = [LpVariable(name = f'dist_cour{i}', cat = LpInteger) for i in range(mcpp.m)]




# partial distance made by each courier
dist_par  = [[LpVariable(name = f'dist{i}_{j}', cat = LpInteger) for j in range(mcpp.n + 2 )] for i in range(mcpp.m)] 

ordering = [[LpVariable(name = f'ord{i}_{j}', lowBound = 0, upBound = 1,cat = LpBinary) for j in range(mcpp.n + 1 )] for i in range(mcpp.m)] 


In [156]:
maximum = LpVariable(name=f'maximum', lowBound = 0, upBound = 10000, cat = LpInteger)
model = LpProblem(name=f'mcpp1', sense=LpMinimize)

In [162]:
#constraints 
#1. each element of X can assume just one value 
for i in range(mcpp.m):
    for j in range(mcpp.n):
        model += lpSum(X[i][j]) == 1 
    
#2. no repetions in the ordering sequences are allowed 
for k in range(1, mcpp.n + 1):
    # for each row
    for i in range(mcpp.m):
        model += lpSum([X[i][j][k] for j in range(mcpp.n)]) <= 1

#3. each package has to be shipped
for j in range(mcpp.n):
    model += lpSum([X[i][j][k]  for k in range(1, mcpp.n + 1) for i in range(mcpp.m)]) == 1

#4. load size has to be respected 
for i in range(mcpp.m):
    for j in range (mcpp.n):
        model += lpSum(lpSum([X[i][j][k] for k in range(1,mcpp.n + 1)]))* mcpp.s[j] <= mcpp.l[i]


#5. need to set an ordering  constraints
for i in range(mcpp.m):
    prec = 0
    for j in range(mcpp.n):
        
            model+= ordering[i][prec] == lpSum(X[i][j][k] for k in range(1,mcpp.n + 1))
            prec = prec + 1

for i in range(mcpp.m):
    for j in range(2,mcpp.n + 1):
        model += ordering[i][j] >= ordering[i][j - 1]


In [148]:
for i in range(mcpp.m):
    model += dist_par[i][0] == lpSum(X[i][j][1]*mcpp.D[mcpp.n][j] for j in range(mcpp.n))
    model += dist_par[i][mcpp.n+1] == lpSum(X[i][j][mcpp.n]*mcpp.D[j][mcpp.n] for j in range(mcpp.n)) 
 
    for k in range(2,mcpp.n+1):
         model += dist_par[i][k - 1] == lpSum(linear_prod(model,And(model,X[i][j2][k-1], X[i][j][k],f'dist{j}_{j2}'),mcpp.D[j2][j],mcpp.D[j2][j],f'prod{j2}{j}') for j2 in range(mcpp.n) for j in range(mcpp.n))
    



In [149]:
for i in range(mcpp.m):
    model += dist_courier[i] == lpSum(dist_par[i])

In [150]:

#obj
for i in range(mcpp.m):
    model += maximum >= lpSum(dist_par[i]) 
model += maximum

c:\Users\shola\AppData\Local\Programs\Python\Python39\lib\site-packages\pulp\pulp.py:1711: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [151]:
status = model.solve()


PulpSolverError: Pulp: Error while executing c:\Users\shola\AppData\Local\Programs\Python\Python39\lib\site-packages\pulp\solverdir\cbc\win\64\cbc.exe

In [ ]:
for var in model.variables():
    print(f"{var.name}: Value = {var.varValue}")


X_0_0_0: Value = 0.0
X_0_0_1: Value = 0.0
X_0_0_2: Value = 0.0
X_0_0_3: Value = 0.0
X_0_0_4: Value = 0.0
X_0_0_5: Value = 1.0
X_0_0_6: Value = 0.0
X_0_1_0: Value = 1.0
X_0_1_1: Value = 0.0
X_0_1_2: Value = 0.0
X_0_1_3: Value = 0.0
X_0_1_4: Value = 0.0
X_0_1_5: Value = 0.0
X_0_1_6: Value = 0.0
X_0_2_0: Value = 1.0
X_0_2_1: Value = 0.0
X_0_2_2: Value = 0.0
X_0_2_3: Value = 0.0
X_0_2_4: Value = 0.0
X_0_2_5: Value = 0.0
X_0_2_6: Value = 0.0
X_0_3_0: Value = 1.0
X_0_3_1: Value = 0.0
X_0_3_2: Value = 0.0
X_0_3_3: Value = 0.0
X_0_3_4: Value = 0.0
X_0_3_5: Value = 0.0
X_0_3_6: Value = 0.0
X_0_4_0: Value = 1.0
X_0_4_1: Value = 0.0
X_0_4_2: Value = 0.0
X_0_4_3: Value = 0.0
X_0_4_4: Value = 0.0
X_0_4_5: Value = 0.0
X_0_4_6: Value = 0.0
X_0_5_0: Value = 1.0
X_0_5_1: Value = 0.0
X_0_5_2: Value = 0.0
X_0_5_3: Value = 0.0
X_0_5_4: Value = 0.0
X_0_5_5: Value = 0.0
X_0_5_6: Value = 0.0
X_1_0_0: Value = 1.0
X_1_0_1: Value = 0.0
X_1_0_2: Value = 0.0
X_1_0_3: Value = 0.0
X_1_0_4: Value = 0.0
X_1_0_5: Valu